# Analysis of Bike Trips in 2019

## Setup
Import all libraries.

In [1]:
!pip install chart_studio

    100% |████████████████████████████████| 81kB 18.3MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import os
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

## Download
Get all available trip data for 2019.

In [3]:
!curl -O "https://s3.amazonaws.com/tripdata/2019[01-10]-citibike-tripdata.csv.zip"
!unzip -o "*.zip"
!rm *.zip


[1/10]: https://s3.amazonaws.com/tripdata/201901-citibike-tripdata.csv.zip --> 201901-citibike-tripdata.csv.zip
--_curl_--https://s3.amazonaws.com/tripdata/201901-citibike-tripdata.csv.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39.5M  100 39.5M    0     0  34.9M      0  0:00:01  0:00:01 --:--:-- 34.9M

[2/10]: https://s3.amazonaws.com/tripdata/201902-citibike-tripdata.csv.zip --> 201902-citibike-tripdata.csv.zip
--_curl_--https://s3.amazonaws.com/tripdata/201902-citibike-tripdata.csv.zip
100 38.5M  100 38.5M    0     0  42.5M      0 --:--:-- --:--:-- --:--:-- 60.5M

[3/10]: https://s3.amazonaws.com/tripdata/201903-citibike-tripdata.csv.zip --> 201903-citibike-tripdata.csv.zip
--_curl_--https://s3.amazonaws.com/tripdata/201903-citibike-tripdata.csv.zip
100 54.5M  100 54.5M    0     0  52.5M      0  0:00:01  0:00:01 --:--:-- 52.1M

[4/10]: https://s3.amazonaws.com/t

## Create DataFrame
Read 2019 trip data into a DataFrame.

In [4]:
files = !ls *.csv
df = pd.concat([pd.read_csv(f, header=0, skip_blank_lines=True) for f in files], keys=files)
df.head()

tripduration                 starttime  \
201901-citibike-tripdata.csv 0           320  2019-01-01 00:01:47.4010   
                             1           316  2019-01-01 00:04:43.7360   
                             2           591  2019-01-01 00:06:03.9970   
                             3          2719  2019-01-01 00:07:03.5450   
                             4           303  2019-01-01 00:07:35.9450   

                                                stoptime  start station id  \
201901-citibike-tripdata.csv 0  2019-01-01 00:07:07.5810            3160.0   
                             1  2019-01-01 00:10:00.6080             519.0   
                             2  2019-01-01 00:15:55.4380            3171.0   
                             3  2019-01-01 00:52:22.6500             504.0   
                             4  2019-01-01 00:12:39.5020             229.0   

                                         start station name  \
201901-citibike-tripdata.csv 0  Central Park West & W 76 St   
                             1        Pershing Square North   
                             2      Amsterdam Ave & W 82 St   
                             3              1 Ave & E 16 St   
                             4               Great Jones St   

                                start station latitude  \
201901-citibike-tripdata.csv 0               40.778968   
                             1               40.751873   
                             2               40.785247   
                             3               40.732219   
                             4               40.727434   

                                start station longitude  end station id  \
201901-citibike-tripdata.csv 0               -73.973747          3283.0   
                             1               -73.977706           518.0   
                             2               -73.976673          3154.0   
                             3               -73.981656          3709.0   
                             4               -73.993790           503.0   

                                      end station name  end station latitude  \
201901-citibike-tripdata.csv 0  W 89 St & Columbus Ave             40.788221   
                             1         E 39 St & 2 Ave             40.747804   
                             2         E 77 St & 3 Ave             40.773142   
                             3         W 15 St & 6 Ave             40.738046   
                             4      E 20 St & Park Ave             40.738274   

                                end station longitude  bikeid    usertype  \
201901-citibike-tripdata.csv 0             -73.970416   15839  Subscriber   
                             1             -73.973442   32723  Subscriber   
                             2             -73.958562   27451  Subscriber   
                             3             -73.996430   21579  Subscriber   
                             4             -73.987520   35379  Subscriber   

                                birth year  gender  
201901-citibike-tripdata.csv 0        1971       1  
                             1        1964       1  
                             2        1987       1  
                             3        1990       1  
                             4        1979       1

In [5]:
df.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,1.811778e+07,1.811760e+07,1.811778e+07,1.811778e+07,1.811760e+07,1.811778e+07,1.811778e+07,1.811778e+07,1.811778e+07,1.811778e+07
mean,9.975303e+02,1.663640e+03,4.073724e+01,-7.398201e+01,1.655954e+03,4.073693e+01,-7.398221e+01,2.931836e+04,1.980243e+03,1.162605e+00
std,1.070981e+04,1.475535e+03,3.049230e-02,1.977536e-02,1.475292e+03,3.037217e-02,1.986180e-02,7.402559e+03,1.208459e+01,5.421212e-01
min,6.100000e+01,7.200000e+01,4.065540e+01,-7.402535e+01,7.200000e+01,4.065540e+01,-7.408364e+01,1.452900e+04,1.857000e+03,0.000000e+00
25%,3.670000e+02,3.820000e+02,4.071755e+01,-7.399530e+01,3.800000e+02,4.071749e+01,-7.399595e+01,2.516500e+04,1.970000e+03,1.000000e+00
50%,6.260000e+02,5.140000e+02,4.073782e+01,-7.398518e+01,5.130000e+02,4.073705e+01,-7.398565e+01,3.067700e+04,1.983000e+03,1.000000e+00
75%,1.101000e+03,3.295000e+03,4.075725e+01,-7.397121e+01,3.295000e+03,4.075715e+01,-7.397137e+01,3.440700e+04,1.990000e+03,1.000000e+00
max,3.812666e+06,3.891000e+03,4.086900e+01,-7.387800e+01,3.891000e+03,4.086900e+01,-7.387800e+01,4.206800e+04,2.003000e+03,2.000000e+00


## Find Most Popular Destinations
Group by `end station name` and plot top ten.

In [13]:
df_end_station = df.groupby(['end station name']).size().nlargest(30).reset_index(name='trips')
bar_end_station = go.Bar(x=df_end_station['end station name'],
                         y=df_end_station['trips'])
layout_end_station = go.Layout(title='Most Popular Destinations')
fig_end_station = go.Figure(data=[bar_end_station],
                            layout=layout_end_station)
iplot(fig_end_station)

## Find Most Popular Starts
Group by `start station name` and plot top ten.

In [14]:
df_start_station = df.groupby(['start station name']).size().nlargest(30).reset_index(name='trips')
bar_start_station = go.Bar(x=df_start_station['start station name'],
                           y=df_start_station['trips'])
layout_start_station = go.Layout(title='Most Popular Starts')
fig_start_station = go.Figure(data=[bar_start_station],
                              layout=layout_start_station)
iplot(fig_start_station)

## Create Bins for Part of Day
Label `part of day` to see if certain hours have more riders.

In [8]:
def bin_part_of_day(hour):
    if hour < 2:
        return 'night'
    if hour < 6:
        return 'before dawn'
    if hour < 10:
        return 'morning'
    if hour < 14:
        return 'midday'
    if hour < 18:
        return 'afternoon'
    if hour < 22:
        return 'evening'
    else:
        return 'night'
    
# Convert `start time` to DateTime
df['starttime'] = df['starttime'].apply(pd.to_datetime)
df['part of day'] = df['starttime'].apply(lambda x: bin_part_of_day(x.hour))
df.head()

tripduration               starttime  \
201901-citibike-tripdata.csv 0           320 2019-01-01 00:01:47.401   
                             1           316 2019-01-01 00:04:43.736   
                             2           591 2019-01-01 00:06:03.997   
                             3          2719 2019-01-01 00:07:03.545   
                             4           303 2019-01-01 00:07:35.945   

                                                stoptime  start station id  \
201901-citibike-tripdata.csv 0  2019-01-01 00:07:07.5810            3160.0   
                             1  2019-01-01 00:10:00.6080             519.0   
                             2  2019-01-01 00:15:55.4380            3171.0   
                             3  2019-01-01 00:52:22.6500             504.0   
                             4  2019-01-01 00:12:39.5020             229.0   

                                         start station name  \
201901-citibike-tripdata.csv 0  Central Park West & W 76 St   
                             1        Pershing Square North   
                             2      Amsterdam Ave & W 82 St   
                             3              1 Ave & E 16 St   
                             4               Great Jones St   

                                start station latitude  \
201901-citibike-tripdata.csv 0               40.778968   
                             1               40.751873   
                             2               40.785247   
                             3               40.732219   
                             4               40.727434   

                                start station longitude  end station id  \
201901-citibike-tripdata.csv 0               -73.973747          3283.0   
                             1               -73.977706           518.0   
                             2               -73.976673          3154.0   
                             3               -73.981656          3709.0   
                             4               -73.993790           503.0   

                                      end station name  end station latitude  \
201901-citibike-tripdata.csv 0  W 89 St & Columbus Ave             40.788221   
                             1         E 39 St & 2 Ave             40.747804   
                             2         E 77 St & 3 Ave             40.773142   
                             3         W 15 St & 6 Ave             40.738046   
                             4      E 20 St & Park Ave             40.738274   

                                end station longitude  bikeid    usertype  \
201901-citibike-tripdata.csv 0             -73.970416   15839  Subscriber   
                             1             -73.973442   32723  Subscriber   
                             2             -73.958562   27451  Subscriber   
                             3             -73.996430   21579  Subscriber   
                             4             -73.987520   35379  Subscriber   

                                birth year  gender part of day  
201901-citibike-tripdata.csv 0        1971       1       night  
                             1        1964       1       night  
                             2        1987       1       night  
                             3        1990       1       night  
                             4        1979       1       night

In [17]:
df_partofday = df.groupby(['part of day']).size().reset_index(name='trips')
bar_partofday = go.Bar(x=df_partofday['part of day'],
                       y=df_partofday['trips'])
layout_partofday = go.Layout(title='Rides by Part of Day')
fig_partofday = go.Figure(data=[bar_partofday],
                          layout=layout_partofday)
iplot(fig_partofday)

## Plot by weekend
Plot to see if there rides by time of day differs during weekdays and weekends.

In [10]:
df['weekday'] = df['starttime'].apply(lambda x: x.dayofweek < 5)
df_weekday_partofday = df.groupby(['weekday','part of day']).size().reset_index(name='trips')

df_wdyonly = df_weekday_partofday[df_weekday_partofday['weekday']]
df_wkdonly = df_weekday_partofday[df_weekday_partofday['weekday'] != True]

bar_df_wdyonly = go.Bar(x=df_wdyonly['part of day'],
                        y=df_wdyonly['trips'])
layout_wdyonly = go.Layout(title='Weekday Rides by Part of Day')
fig_wdyonly = go.Figure(data=[bar_df_wdyonly],
                        layout=layout_wdyonly)

bar_df_wkdonly = go.Bar(x=df_wkdonly['part of day'],
                        y=df_wkdonly['trips'])
layout_wkdonly = go.Layout(title='Weekend Rides by Part of Day')
fig_wkdonly = go.Figure(data=[bar_df_wkdonly],
                        layout=layout_wkdonly)

iplot(fig_wdyonly)
iplot(fig_wkdonly)

## Find Most Popular Month by Rides
Chart rides by month.

In [11]:
df['month of year'] = df['starttime'].apply(lambda x: x.month)
df['month name'] = df['starttime'].apply(lambda x: x.strftime('%b'))
df_monthofyr = df.groupby(['month of year','month name']).size().reset_index(name='trips')
bar_monthofyr = go.Bar(x=df_monthofyr['month name'],
                       y=df_monthofyr['trips'])
layout_monthofyr = go.Layout(title='Rides by Month')
fig_monthofyr = go.Figure(data=[bar_monthofyr],
                          layout=layout_monthofyr)
iplot(fig_monthofyr)